#Install libary


In [1]:
# !pip install black[jupyter]
!pip install -U tensorflow-addons
!pip install facenet-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 9.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 8.0 MB/s 


#Import and Initial Mount Disk

In [2]:
import os
# Mount drive
from google.colab import drive
drive.mount("/content/drive")
path = "/content/drive/My Drive/Colab Notebooks/FaceMaskRecognize"
os.chdir(path)

import cv2
import io
import pickle
import math
from scipy.spatial.distance import cosine
from sklearn.preprocessing import normalize
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torchvision.transforms as transforms
from train.Net import InceptionResNetV1
from train.FaceNet import FaceNetModel,call_instance_FaceNet_with_custom, call_instance_FaceNet_without_custom, call_instance_FaceNet_with_last_isDense,convert_train_model_to_embedding
from train.Classify import Classify
from tool.FormatFunction import FormatFunction
from tool.FileFunction import FileFunction
from tool.GlobalValue import GlobalValue
from facenet_pytorch import InceptionResnetV1


Mounted at /content/drive


#Format Code

In [ ]:
# !black TensorFlow.ipynb

# Work with tensorflow

## Create Constant Tensor

In [ ]:
# # Create constant tensor
# scalar = tf.constant(value=[23, 31, 3123])
# print(scalar)
# # Check for the dimension of tensor
# print(scalar.ndim)
# # Create constant tensor with 2 dimension
# matrix = tf.constant([[12, 32], [31, 32], [3123, 1], [1, 4]])
# print(matrix)
# # Create constant tensor with 3 dimension
# matrix = tf.constant([[[12, 32], [31, 32]], [[3123, 1], [1, 4]]])
# print(matrix)

##Create Tensor with Variable and check how they work

In [ ]:
# # Create tensor with tf.Variable
# changeableTensor = tf.Variable([10, 12])
# unchangeableTensor = tf.constant([10, 10])
# changeableTensor[0].assign(3)
# # unchangeableTensor[0].assign(4) cannot assign like Tensorflow
# print(changeableTensor)
# print(unchangeableTensor)

##Create Random Tensor with tf.random

In [ ]:
# random1 = tf.random.Generator.from_seed(10)
# random1 = random1.normal([3, 3])
# random2 = tf.random.Generator.from_seed(10)
# random2 = random2.normal([3, 3])
# print(random1)
# print(random2)
# print("----------------------Shuffle tensor with random shuffle---------------")
# not_shuffle = tf.constant([[12, 3], [4, 3], [13, 15], [1, 4]])
# print(tf.random.shuffle(not_shuffle))


##Another way to create Tensor

In [ ]:
# #All is 1
# tf.ones([4,7])

In [ ]:
# #All is zero
# tf.zeros([4,8])

##Create tensor with numpy

In [ ]:
# print("--------------------Create tensor from numpy array--------------------------")
# arrayA = np.arange(start=1, stop=13, dtype=int)
# matrixFromA = tf.constant(arrayA, shape=(2, 3, 2))
# print(matrixFromA)

##Tensor Attribute

In [ ]:
# a Rank 4 tensor
tensor4D = tf.constant(np.arange(start = 1, stop = 73),shape=[3,2,3,4])
print("---------------tensor-------------\n", tensor4D)
print("---------------indexing Tensor-----------\n", tensor4D[:,:,0])
print("----------------Get shape tensor------------\n", tensor4D.shape)
print("----------------Dimension------------------\n", tensor4D.ndim)
print("---------------Size------------\n", tf.size(tensor4D).numpy())
print("-------------data type------------\n", tensor4D.dtype)

---------------tensor-------------
 tf.Tensor(
[[[[ 1  2  3  4]
   [ 5  6  7  8]
   [ 9 10 11 12]]

  [[13 14 15 16]
   [17 18 19 20]
   [21 22 23 24]]]


 [[[25 26 27 28]
   [29 30 31 32]
   [33 34 35 36]]

  [[37 38 39 40]
   [41 42 43 44]
   [45 46 47 48]]]


 [[[49 50 51 52]
   [53 54 55 56]
   [57 58 59 60]]

  [[61 62 63 64]
   [65 66 67 68]
   [69 70 71 72]]]], shape=(3, 2, 3, 4), dtype=int64)
---------------indexing Tensor-----------
 tf.Tensor(
[[[ 1  2  3  4]
  [13 14 15 16]]

 [[25 26 27 28]
  [37 38 39 40]]

 [[49 50 51 52]
  [61 62 63 64]]], shape=(3, 2, 4), dtype=int64)
----------------Get shape tensor------------
 (3, 2, 3, 4)
----------------Dimension------------------
 4
---------------Size------------
 72
-------------data type------------
 <dtype: 'int64'>


##Reshape tensor

In [ ]:
# tensor4D = tf.zeros(shape=[3,2,3,4])
# tensor2D = tf.reshape(tensor4D, shape = [8,9])
# print(tensor2D)

## Extend tensor

In [ ]:
# tensor2D = tf.zeros(shape = [4,5])
# print("Original Tensor\n", tensor2D)
# tensor3D = tf.expand_dims(tensor2D, axis = -1)
# print("New Tensor\n", tensor3D)

# Manupilating tensor

## Basic operation

In [ ]:
# tensor = tf.constant([[10,4],[12,5]])
# # Use build-in operation
# print("addition\n",tensor+10)
# print("subtraction\n",tensor-10)
# print("multiplication\n",tensor*10)
# print("division\n",tensor/10)

# # Use function
# print("________________________________")
# print("addition\n",tf.add(tensor,10))
# print("subtraction\n",tf.subtract(tensor,10))
# print("multiplication\n",tf.multiply(tensor,10))
# print("divide\n",tf.divide(tensor,10))

## Matrix Multiplication

In [ ]:
# first_tensor = tf.constant([[10,4,6],
#                             [12,5,3],
#                             [32,4,1]])
# second_tensor = tf.constant([[5,6],
#                              [12,5],
#                              [5,3]])
# print(tf.matmul(first_tensor,second_tensor))

# Build train model

## Init value

In [ ]:
# global_value = GlobalValue(image_size=[80,80], batch_size = 48, shuffle_size = 512, ratio_train = 0.8, epochs = 40, small_epochs = 2)
# format_function = FormatFunction(global_value)

## Test Train Model tensorflow

In [ ]:
# train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)

# # Build your input pipelines
# train_dataset = train_dataset.map(format_function.format_gray_image)
# train_dataset = train_dataset.shuffle(1024).batch(global_value.BATCH_SIZE)
# test_dataset = test_dataset.map(format_function.format_gray_image)
# test_dataset = test_dataset.batch(global_value.BATCH_SIZE)

# # The embedding model
# input_shape = [global_value.IMAGE_SIZE[0], global_value.IMAGE_SIZE[1], 3]
# face_net_model = call_instance_FaceNet_with_last_isDense(input_shape,10)


# # Compile the model
# face_net_model.compile(
#     optimizer=tf.keras.optimizers.Adam(0.001),
#     loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
#     )


# # Train the network
# history = face_net_model.fit(
#       train_dataset,
#       epochs=global_value.EPOCHS
#       )
# face_net_model.save("save_model/modify_train_mnist")


# # Evaluate the network
# results = face_net_model.predict(test_dataset)

# # Save test embeddings for visualization in projector
# np.savetxt("vecs_48.tsv", results, delimiter='\t')

# out_m = io.open('meta_48.tsv', 'w', encoding='utf-8')
# for img, labels in tfds.as_numpy(test_dataset):
#     [out_m.write(str(x) + "\n") for x in labels]
# out_m.close()


# try:
#   from google.colab import files
#   files.download('vecs_48.tsv')
#   files.download('meta_48.tsv')
# except:
#   pass

## Save and Load model


In [ ]:
# # save model
# face_net_model.save("save_model/number_mnist_48")
# # save loss value
# loss_array = history.history['loss']
# with open("src/loss/mnist_loss.pkl", 'wb') as file:
#     pickle.dump(loss_array, file)

# reconstruct_model = tf.keras.models.load_model("save_model/modify_train_mnist")
# reconstruct_model = convert_train_model_to_embedding(reconstruct_model)
# results = reconstruct_model.predict(test_dataset)
# # Save test embeddings for visualization in projector
# np.savetxt("vecs_48.tsv", results, delimiter='\t')

# out_m = io.open('meta_48.tsv', 'w', encoding='utf-8')
# for img, labels in tfds.as_numpy(test_dataset):
#     [out_m.write(str(x) + "\n") for x in labels]
# out_m.close()


# try:
#   from google.colab import files
#   files.download('vecs_48.tsv')
#   files.download('meta_48.tsv')
# except:
#   pass

## Encoding Image

In [ ]:
# train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)
# encoding_dic  = dict()

# for i in range (10):
#     data_set = train_dataset.filter(lambda image,label: tf.equal(label,i))
#     data_set = data_set.map(format_function.format_gray_image)
#     data_set = data_set.batch(10)  
#     encodes = reconstruct_model.predict(data_set)
#     if encodes.any():
#         encodes = np.average(encodes, axis=0 )
#         encoding_dic[i] = encodes

# with open("encodings/encoding_train_cross_entropy.pkl", 'wb') as file:
#     pickle.dump(encoding_dic, file)

## Predict

In [ ]:
# def load_pickle(path):
#     with open(path, 'rb') as f:
#         encoding_dict = pickle.load(f)
#     return encoding_dict

# recognition_t=5

# def detect(image, encoder,encoding_dict):
#     image = tf.expand_dims(image,0)
#     encode = encoder(image)
#     name = "unknow"
#     distance = float("inf")
#     for db_name, db_encode in encoding_dict.items():
#             dist = cosine(db_encode, encode)
#             if dist < recognition_t and dist < distance:
#                 name = db_name
#                 distance = dist

#     return name

# train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)
# train_dataset = train_dataset.map(format_function.format_gray_image)
# test_dataset = test_dataset.map(format_function.format_gray_image)
# combined_dataset = train_dataset.concatenate(test_dataset)
# combined_dataset = combined_dataset


# total = 0
# right_choice = 0
# i = 0
# total_image = len(combined_dataset)
# # plt.figure(figsize=(3*(total_image/10), 30))
# for image, label in combined_dataset:
#     predict_label = detect(image, face_net_model, load_pickle("encodings/encoding_train_cross_entropy.pkl"))
#     print("predict: {} |||||| actual: {}".format(predict_label, label))
#     # plt.subplot(total_image/10,10,i+1)
#     # plt.imshow(image.numpy())
#     # plt.title("predict: {} |||||| actual: {}".format(predict_label, label))
#     # plt.axis("off")
#     total = total + 1
#     if(tf.equal(predict_label,label)):
#         right_choice = right_choice + 1
#     i = i+1
# print(total, right_choice)

# Test embedding

In [ ]:
# global_value = GlobalValue(image_size=[110,110], batch_size = 48, shuffle_size = 512, ratio_train = 0.8, epochs = 40, small_epochs = 2)
# format_function = FormatFunction(global_value)
# # Load network
# face_net_model = tf.keras.models.load_model("save_model/face_recognize_entropy4")
# face_net_model = convert_train_model_to_embedding(face_net_model)
# #Preprocess data
# test_dataset = tf.data.Dataset.list_files("dataset/10_person/*/*",shuffle=False)
# test_dataset = test_dataset.map(format_function.get_label_as_string, num_parallel_calls=tf.data.AUTOTUNE)
# test_dataset = test_dataset.map(format_function.process_image, num_parallel_calls=tf.data.AUTOTUNE)
# test_dataset = test_dataset.filter(lambda image, label: tf.math.not_equal(tf.size(image), 0))
# test_dataset = test_dataset.batch(20)
# # Evaluate the network
# results = face_net_model.predict(test_dataset)

# # Save test embeddings for visualization in projector
# np.savetxt("vecs.tsv", results, delimiter='\t')

# out_m = io.open('meta.tsv', 'w', encoding='utf-8')
# for img, labels in tfds.as_numpy(test_dataset):
#     [out_m.write(str(x) + "\n") for x in labels]
# out_m.close()


# try:
#   from google.colab import files
#   files.download('vecs.tsv')
#   files.download('meta.tsv')
# except:
#   pass

# Test embedding pytorch


In [ ]:
global_value = GlobalValue(image_size=[110,110], batch_size = 48, shuffle_size = 512, ratio_train = 0.8, epochs = 40, small_epochs = 2)
format_function = FormatFunction(global_value)
file_function = FileFunction()
transform = transforms.ToTensor()
# Load network
face_net_model = InceptionResnetV1(pretrained='casia-webface').eval()
#Preprocess data
list_path = file_function.getPath(os.path.join(os.getcwd(), "dataset", "10_person"))
list_label = [x.split(os.path.sep)[-2] for x in list_path]
list_image = []
for each_path in list_path:
  image = format_function.open_and_process_image_Pillow(each_path)
  image = transform(image)
  image = image.float()
  list_image.append(image)
list_image_as_tensor = torch.stack(list_image,0)

# Evaluate the network
results = face_net_model(list_image_as_tensor).detach()
results = normalize(results)


# Save test embeddings for visualization in projector
np.savetxt("vecs.tsv", results, delimiter='\t')

out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for label in list_label:
    out_m.write(str(label) + "\n") 
out_m.close()


try:
  from google.colab import files
  files.download('vecs.tsv')
  files.download('meta.tsv')
except:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Evaluate

In [ ]:
#Init
global_value = GlobalValue(image_size=[110,110], batch_size = 48, shuffle_size = 512, ratio_train = 0.8, epochs = 40, small_epochs = 2)
format_function = FormatFunction(global_value)

for i in range(49,50):
  print("evaluate epoch: ", i)
  face_net_model = tf.keras.models.load_model("save_model/face_recognize_entropy{}".format(i))
  face_net_model = convert_train_model_to_embedding(face_net_model)
  classify = Classify(face_net_model, format_function)

  #Get embedding database
  # database_embedding = classify.embedding_all_data_by_directory(os.path.join(os.getcwd(),"dataset","lfw_align"))
  # classify.save_embedding_to_file(database_embedding, os.path.join(os.getcwd(),"src","encodings","encode_lfw_entropy{}(test).pkl".format(i)))
  database_embedding = classify.load_embedding_from_file(os.path.join(os.getcwd(),"src","encodings","encode_lfw_entropy{}(test).pkl".format(i)))
  #Preprocess data
  test_dataset = tf.data.Dataset.list_files(["dataset/lfw_mask/*/*"],shuffle=False)
  test_dataset = test_dataset.map(format_function.get_label_as_string, num_parallel_calls=tf.data.AUTOTUNE)
  test_dataset = test_dataset.map(format_function.process_image, num_parallel_calls=tf.data.AUTOTUNE)
  test_dataset = test_dataset.batch(20)

  # Accuracy
  accuracy = classify.evaluate(test_dataset, database_embedding)
  print(accuracy)
  with open("src/accuracy/face_recognize_entropy(test).txt", "a") as file_object:
    file_object.write("\n")
    file_object.write("epoch {}, accuracy {}".format(i, accuracy))

evaluate epoch:  49


 21%|**        | 123/592 [02:49<11:27,  1.47s/it]

In [ ]:
face_net_model = tf.keras.models.load_model("save_model/face_recognize_entropy39")
face_net_model.save_weights("model39.h5")

# Predict one image